### Environment

In [1]:
import pandas as pd
from datasets import Dataset, DatasetDict, load_dataset

c:\Users\baech_6wlvk\anaconda3\envs\torch23\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Load dataset

In [2]:
data_files = {"../dataset_jit/train_jit.csv"}
dataset =  load_dataset("csv", data_files=data_files, delimiter="\t")

In [3]:
dataset

DatasetDict({
    train: Dataset({
        features: ['ko', 'je'],
        num_rows: 160000
    })
})

### Tokenizer

In [4]:
import torch
from tqdm import tqdm
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

tokenizer = PreTrainedTokenizerFast.from_pretrained('hyunwoongko/kobart')
model = BartForConditionalGeneration.from_pretrained('hyunwoongko/kobart')

c:\Users\baech_6wlvk\anaconda3\envs\torch23\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [5]:
examples= {'je': dataset['train']["je"], 
           'ko': dataset['train']["ko"]}

In [6]:
def preprocess_function(example):
    inputs = example['ko']
    targets = example['je']
    model_inputs = tokenizer(inputs, max_length=128, padding='max_length', truncation=True)

    # KoBART의 입력과 출력을 설정합니다.
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, padding='max_length', truncation=True)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, remove_columns=dataset["train"].column_names,batched=True)


In [7]:
print( '원 데이터    :', dataset['train'][10]['je'] )
print( '처리 후 데이터:',  tokenizer.convert_ids_to_tokens(tokenized_datasets['train'][10]['input_ids']))
print( '토큰화       :', tokenized_datasets['train'][10]['input_ids'] )

print('\n')
print( '원 데이터    :', dataset['train'][10]['ko'] )
print( '처리 후 데이터:', tokenizer.convert_ids_to_tokens(tokenized_datasets['train'][10]['input_ids']) )
print( '토큰화       :', tokenized_datasets['train'][10]['labels'] )

원 데이터    : 예 .
처리 후 데이터: ['<s>', '▁예', '▁.', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<

In [8]:
import evaluate
import numpy as np
metric = evaluate.load("sacrebleu")

def compute_metrics_bleu(eval_preds):
    preds, labels = eval_preds
    
    if isinstance(preds, tuple):
        preds = preds[0]
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    
    return result

In [9]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 160000
    })
})

In [10]:
formatted_train_df = pd.DataFrame([{
    "input_ids": np.array(fd["input_ids"], dtype=np.uint16),
    "attention_mask": np.array(fd["attention_mask"], dtype=np.uint8),
    "labels": np.array(fd["labels"], dtype=np.uint16)
} for fd in tokenized_datasets["train"]])

#formatted_valid_df = pd.DataFrame([{
#    "input_ids": np.array(fd["input_ids"], dtype=np.uint16),
#    "attention_mask": np.array(fd["attention_mask"], dtype=np.uint8),
#    "labels": np.array(fd["labels"], dtype=np.uint16)
#} for fd in tokenized_datasets["valid"]])

In [11]:
train_dataset = Dataset.from_pandas(formatted_train_df)
#valid_dataset = Dataset.from_pandas(formatted_valid_df)

In [12]:
train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 160000
})

In [13]:
train_test_split = train_dataset.train_test_split(test_size=0.1)
dataset_dict = DatasetDict({
    'train': train_test_split['train'],
    'valid': train_test_split['test']
})

In [14]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 144000
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 16000
    })
})

In [15]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="result/bleu",                         
    evaluation_strategy="epoch",                        
    learning_rate=2e-5,                               
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,                  
    weight_decay=0.01,                                
    save_total_limit=3,                               
    num_train_epochs=5,                                 
    predict_with_generate=True,                        
)

trainer = Seq2SeqTrainer(
    model=model,                        
    args=training_args,                 
    train_dataset=dataset_dict['train'],         
    eval_dataset=dataset_dict['valid'],
    compute_metrics=compute_metrics_bleu
)

trainer.train()

  0%|          | 0/11250 [00:00<?, ?it/s]c:\Users\baech_6wlvk\anaconda3\envs\torch23\lib\site-packages\transformers\models\bart\modeling_bart.py:587: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
  4%|▍         | 500/11250 [21:31<7:50:38,  2.63s/it]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 1}


{'loss': 0.8826, 'grad_norm': 0.5195364952087402, 'learning_rate': 1.9111111111111113e-05, 'epoch': 0.22}


  9%|▉         | 1000/11250 [43:03<7:20:44,  2.58s/it]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 1}


{'loss': 0.1574, 'grad_norm': 0.4221462607383728, 'learning_rate': 1.8222222222222224e-05, 'epoch': 0.44}


 13%|█▎        | 1500/11250 [1:04:40<7:22:34,  2.72s/it]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 1}


...

 98%|█████████▊| 11000/11250 [6:54:45<08:47,  2.11s/it]Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 1}


{'loss': 0.0753, 'grad_norm': 0.3156682252883911, 'learning_rate': 4.444444444444445e-07, 'epoch': 4.89}


100%|██████████| 11250/11250 [7:03:33<00:00,  2.11s/it]c:\Users\baech_6wlvk\anaconda3\envs\torch23\lib\site-packages\transformers\generation\utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
                                                       
100%|██████████| 11250/11250 [7:05:50<00:00,  2.27s/it]

{'eval_loss': 0.0818123146891594, 'eval_bleu': 31.530046175977446, 'eval_runtime': 136.7061, 'eval_samples_per_second': 117.039, 'eval_steps_per_second': 1.829, 'epoch': 5.0}
{'train_runtime': 25550.1475, 'train_samples_per_second': 28.18, 'train_steps_per_second': 0.44, 'train_loss': 0.12863388264973957, 'epoch': 5.0}


TrainOutput(global_step=11250, training_loss=0.12863388264973957, metrics={'train_runtime': 25550.1475, 'train_samples_per_second': 28.18, 'train_steps_per_second': 0.44, 'total_flos': 5.48762812416e+16, 'train_loss': 0.12863388264973957, 'epoch': 5.0})

In [30]:
trainer.evaluate()

c:\Users\baech_6wlvk\anaconda3\envs\torch23\lib\site-packages\transformers\generation\utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


























































































































































































































































100%|██████████| 250/250 [03:14<00:00,  1.29it/s]


{'eval_loss': 0.0818123146891594,
 'eval_bleu': 31.530046175977446,
 'eval_runtime': 198.6908,
 'eval_samples_per_second': 80.527,
 'eval_steps_per_second': 1.258,
 'epoch': 5.0}

In [32]:
trainer.save_pretrained("./result/bleu")

AttributeError: 'Seq2SeqTrainer' object has no attribute 'save_pretrained'

In [33]:
# 모델 저장
model.save_pretrained('./bart_bleu')
tokenizer.save_pretrained('./bart_bleu')


AttributeError: 'RegressionMetric' object has no attribute 'save_pretrained'

Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='www.comet.com', port=443): Read timed out. (read timeout=10)")': /clientlib/status-report/update
Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001ECB95933A0>: Failed to resolve 'www.comet.com' ([Errno 11001] getaddrinfo failed)")': /clientlib/status-report/update
